In [1]:
import pandas as pd
import config
import re
# Assuming your CSV file is named 'your_file.csv'
# Replace 'your_file.csv' with the actual filename if different
df = pd.read_csv('data/Waterloo.csv')

In [2]:
distinct_listing_count = df['listing_id'].nunique()
total_row_count = len(df)

print(f"Distinct Listing IDs: {distinct_listing_count}")
print(f"Total Rows: {total_row_count}")

Distinct Listing IDs: 20
Total Rows: 20


In [4]:
df_silver = df[['listing_id', 'listing']]
df_silver.head(20)

,listing_id,listing
0,40638134,"194 Samuel Street , Kitchener - Auditorium"
1,40670040,"18 Tweed Street , Waterloo"
2,40655626,"323 Pastern Trail , Waterloo"
3,40684147,"578 Dogwood Lane , Waterloo"
4,40676379,"502 Manchester Road , Kitchener - Rosemount"
5,40681837,"34 Dietz Avenue S , Waterloo"
6,40681526,"A - 586 Mount Anne Drive , Waterloo"
7,********,(Address display requires re-validate password...
8,40683302,"196 Edwin Street , Kitchener - Fairfield"
9,40680573,"92 Belleview Avenue , Kitchener - Rosemount"


In [5]:
def clean_listing_and_extract_new_element(listing):
    if 'Waterloo' in listing:
        city = 'Waterloo'
    elif 'Kitchener' in listing:
        city = 'Kitchener'
    else:
        city = None
    
    new_element = None
    if city:
        parts = re.split(rf'({city})\s*-\s*', listing)
        if len(parts) > 1:
            cleaned_listing = parts[0] + city
            new_element = parts[2] if len(parts) > 2 else None
        else:
            cleaned_listing = listing
    else:
        cleaned_listing = listing
    
    return cleaned_listing, new_element

df_silver[['cleaned_listing', 'neighborhood_hs']] = df_silver['listing'].apply(lambda x: pd.Series(clean_listing_and_extract_new_element(x)))
df_silver.head(10)

/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_4428/1762521065.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver[['cleaned_listing', 'neighborhood_hs']] = df_silver['listing'].apply(lambda x: pd.Series(clean_listing_and_extract_new_element(x)))
/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_4428/1762521065.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver[['cleaned_listing', 'neighborhood_hs']] = df_silver['listing'].apply(lambda x: pd.Series(clean_listing_a

,listing_id,listing,cleaned_listing,neighborhood_hs
0,40638134,"194 Samuel Street , Kitchener - Auditorium","194 Samuel Street , Kitchener",Auditorium
1,40670040,"18 Tweed Street , Waterloo","18 Tweed Street , Waterloo",None
2,40655626,"323 Pastern Trail , Waterloo","323 Pastern Trail , Waterloo",None
3,40684147,"578 Dogwood Lane , Waterloo","578 Dogwood Lane , Waterloo",None
4,40676379,"502 Manchester Road , Kitchener - Rosemount","502 Manchester Road , Kitchener",Rosemount
5,40681837,"34 Dietz Avenue S , Waterloo","34 Dietz Avenue S , Waterloo",None
6,40681526,"A - 586 Mount Anne Drive , Waterloo","A - 586 Mount Anne Drive , Waterloo",None
7,********,(Address display requires re-validate password...,(Address display requires re-validate password...,-
8,40683302,"196 Edwin Street , Kitchener - Fairfield","196 Edwin Street , Kitchener",Fairfield
9,40680573,"92 Belleview Avenue , Kitchener - Rosemount","92 Belleview Avenue , Kitchener",Rosemount


In [6]:
from opencage.geocoder import OpenCageGeocode

key = config.opencage_api
geocoder = OpenCageGeocode(key)

In [ ]:
# query = "469 Laurel Gate Dr, Waterloo"
# results = geocoder.geocode(query)
# components = results[0]['components']

In [7]:
def address_cleaning(input_string):
    match = re.search(r"-\s*(.*)", input_string)
    if match:
        result = match.group(1)
    else:
        result = input_string
    return result

In [8]:
def extract_location_details(component):
  neighbourhood = None
  postcode = None

  if 'neighborhood' in component:
    neighbourhood = component['neighborhood']
  if 'postcode' in component:
    postcode = component['postcode']

  return {'neighbourhood': neighbourhood, 'postcode': postcode}

In [9]:
df_silver['listing'] = df_silver['cleaned_listing'].apply(address_cleaning)
df_silver.drop(columns=['cleaned_listing'], inplace=True)
df_silver.head(10)

/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_4428/4213125178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver['listing'] = df_silver['cleaned_listing'].apply(address_cleaning)
/var/folders/dc/n7cp43k94blc5lmsv8p725pc0000gn/T/ipykernel_4428/4213125178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_silver.drop(columns=['cleaned_listing'], inplace=True)


,listing_id,listing,neighborhood_hs
0,40638134,"194 Samuel Street , Kitchener",Auditorium
1,40670040,"18 Tweed Street , Waterloo",None
2,40655626,"323 Pastern Trail , Waterloo",None
3,40684147,"578 Dogwood Lane , Waterloo",None
4,40676379,"502 Manchester Road , Kitchener",Rosemount
5,40681837,"34 Dietz Avenue S , Waterloo",None
6,40681526,"586 Mount Anne Drive , Waterloo",None
7,********,"validate password) , Waterloo",-
8,40683302,"196 Edwin Street , Kitchener",Fairfield
9,40680573,"92 Belleview Avenue , Kitchener",Rosemount


In [10]:
# df_silver = df_silver.head(200)
df_silver.head(5)

,listing_id,listing,neighborhood_hs
0,40638134,"194 Samuel Street , Kitchener",Auditorium
1,40670040,"18 Tweed Street , Waterloo",None
2,40655626,"323 Pastern Trail , Waterloo",None
3,40684147,"578 Dogwood Lane , Waterloo",None
4,40676379,"502 Manchester Road , Kitchener",Rosemount


In [11]:
len(df_silver)

20

In [12]:
final_dict = {}
for index, row in df_silver.iterrows():
    listing_id = row['listing_id']
    # addy = address_cleaning(row['listing'])
    addy = row['listing']
    # print(f"Listing ID: {listing_id}, Description: {addy}")
    results = geocoder.geocode(addy)
    try:
        components = results[0]['components']
        dict1 = extract_location_details(components)
        dict2 = results[0]['geometry']
        final = {**dict1, **dict2}
        final_dict[listing_id] = final
    except Exception as e:
        # Catch any other exceptions and log the error
        print(f"An unexpected error occurred: {e}")
        print(addy)
        final_dict[listing_id] = {'postcode': None, 
                                  'neighbourhood': None, 
                                  'lat': None, 
                                  'lng':None}
    

In [13]:
final_df = pd.DataFrame.from_dict(final_dict, orient='index')
final_df.reset_index(inplace=True)
final_df.rename(columns={'index': 'listing_id'}, inplace=True)

In [14]:
final_table = df_silver.merge(final_df[['listing_id', 'postcode', 'neighbourhood', 'lat', 'lng']], on='listing_id', how='left')
final_table = final_table[['listing_id','listing', 'postcode', 'neighbourhood', 'neighborhood_hs' , 'lat', 'lng']]

In [15]:
final_table

,listing_id,listing,postcode,neighbourhood,neighborhood_hs,lat,lng
0,40638134,"194 Samuel Street , Kitchener",None,None,Auditorium,43.448782,-80.475351
1,40670040,"18 Tweed Street , Waterloo",N2J 1V5,None,None,43.466885,-80.517622
2,40655626,"323 Pastern Trail , Waterloo",None,None,None,43.492516,-80.491288
3,40684147,"578 Dogwood Lane , Waterloo",N2L 4Z9,None,None,43.486674,-80.559675
4,40676379,"502 Manchester Road , Kitchener",N2B 1J3,None,Rosemount,43.459602,-80.454399
5,40681837,"34 Dietz Avenue S , Waterloo",N2L 1N5,None,None,43.458836,-80.532394
6,40681526,"586 Mount Anne Drive , Waterloo",None,None,None,43.492931,-80.546483
7,********,"validate password) , Waterloo",None,None,-,42.492760,-92.342960
8,40683302,"196 Edwin Street , Kitchener",None,None,Fairfield,43.466166,-80.488399
9,40680573,"92 Belleview Avenue , Kitchener",N2B 1G7,None,Rosemount,43.453151,-80.462850


In [85]:
final_table.to_csv('house_address_postal_codes_new.csv', index=False)